In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('resaleflatprices.csv')
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0
...,...,...,...,...,...,...,...,...,...,...,...
167164,2023-05,YISHUN,EXECUTIVE,827,YISHUN ST 81,01 TO 03,154.0,Maisonette,1987,63 years 06 months,829000.0
167165,2023-05,YISHUN,EXECUTIVE,824,YISHUN ST 81,07 TO 09,146.0,Maisonette,1987,63 years 06 months,855000.0
167166,2023-07,YISHUN,EXECUTIVE,837,YISHUN ST 81,01 TO 03,146.0,Maisonette,1988,63 years 07 months,860000.0
167167,2023-01,YISHUN,MULTI-GENERATION,666,YISHUN AVE 4,07 TO 09,179.0,Multi Generation,1987,63 years 11 months,1080000.0


In [3]:
max_price_town = df.groupby('town')['resale_price'].max()
max_price_town

town
ANG MO KIO         1200000.0
BEDOK              1110888.0
BISHAN             1450000.0
BUKIT BATOK        1088000.0
BUKIT MERAH        1500000.0
BUKIT PANJANG      1020000.0
BUKIT TIMAH        1300000.0
CENTRAL AREA       1480000.0
CHOA CHU KANG       900000.0
CLEMENTI           1280000.0
GEYLANG            1105000.0
HOUGANG            1080000.0
JURONG EAST        1020000.0
JURONG WEST         920000.0
KALLANG/WHAMPOA    1300888.0
MARINE PARADE      1080000.0
PASIR RIS          1238000.0
PUNGGOL            1220000.0
QUEENSTOWN         1418000.0
SEMBAWANG           810000.0
SENGKANG            945000.0
SERANGOON          1150000.0
TAMPINES           1025000.0
TOA PAYOH          1460000.0
WOODLANDS          1150000.0
YISHUN             1080000.0
Name: resale_price, dtype: float64

In [4]:
max_price_by_block = df.groupby(['street_name','block'])['resale_price'].max()
max_price_by_block

street_name    block
ADMIRALTY DR   353A     625000.0
               353B     588000.0
               353C     620000.0
               354A     560000.0
               354B     590000.0
                          ...   
YUNG SHENG RD  181      590000.0
               182      582000.0
               183      570000.0
               184      582000.0
ZION RD        87       980000.0
Name: resale_price, Length: 9512, dtype: float64

In [5]:
street_names = max_price_by_block.index.tolist()
street_names

[('ADMIRALTY DR', '353A'),
 ('ADMIRALTY DR', '353B'),
 ('ADMIRALTY DR', '353C'),
 ('ADMIRALTY DR', '354A'),
 ('ADMIRALTY DR', '354B'),
 ('ADMIRALTY DR', '354C'),
 ('ADMIRALTY DR', '354D'),
 ('ADMIRALTY DR', '356A'),
 ('ADMIRALTY DR', '356B'),
 ('ADMIRALTY DR', '356C'),
 ('ADMIRALTY DR', '357'),
 ('ADMIRALTY DR', '357A'),
 ('ADMIRALTY DR', '357B'),
 ('ADMIRALTY DR', '357C'),
 ('ADMIRALTY DR', '359'),
 ('ADMIRALTY DR', '359A'),
 ('ADMIRALTY DR', '359B'),
 ('ADMIRALTY DR', '359C'),
 ('ADMIRALTY DR', '360A'),
 ('ADMIRALTY DR', '360B'),
 ('ADMIRALTY DR', '360C'),
 ('ADMIRALTY DR', '467'),
 ('ADMIRALTY DR', '467A'),
 ('ADMIRALTY DR', '467B'),
 ('ADMIRALTY DR', '468A'),
 ('ADMIRALTY DR', '468B'),
 ('ADMIRALTY DR', '468C'),
 ('ADMIRALTY DR', '468D'),
 ('ADMIRALTY DR', '469'),
 ('ADMIRALTY DR', '469A'),
 ('ADMIRALTY DR', '469B'),
 ('ADMIRALTY LINK', '401'),
 ('ADMIRALTY LINK', '402'),
 ('ADMIRALTY LINK', '403'),
 ('ADMIRALTY LINK', '404'),
 ('ADMIRALTY LINK', '405'),
 ('ADMIRALTY LINK', '482'),

In [6]:
all_block_names = []
for name in street_names:
    block_name = name[1] + ' ' + name[0]
    all_block_names.append(block_name)

len(all_block_names)

9512

# Requesting long lat from gmaps

In [14]:
import requests
from dotenv import load_dotenv
import os
load_dotenv()
import time

In [15]:
API_KEY = os.getenv("API_KEY")

In [20]:
coords = []
url = 'https://maps.googleapis.com/maps/api/geocode/json'

count = 0

for name in all_block_names:
    keys = {'address': f'{name} , Singapore', 'key': API_KEY}
    r = requests.get(url,params=keys)

    result_dic = r.json()
    one_point = result_dic['results'][0]['geometry']['location']
    one_point_grouped = [one_point['lng'], one_point['lat']]
    
    
    point = { 
        'properties.place': name,
        'geometry.coordinates': one_point_grouped
    }

    coords.append(point)
    # time.sleep(1)

    print(f"loop {count}")
    count += 1

coords

loop 0
loop 1
loop 2
loop 3
loop 4
loop 5
loop 6
loop 7
loop 8
loop 9
loop 10
loop 11
loop 12
loop 13
loop 14
loop 15
loop 16
loop 17
loop 18
loop 19
loop 20
loop 21
loop 22
loop 23
loop 24
loop 25
loop 26
loop 27
loop 28
loop 29
loop 30
loop 31
loop 32
loop 33
loop 34
loop 35
loop 36
loop 37
loop 38
loop 39
loop 40
loop 41
loop 42
loop 43
loop 44
loop 45
loop 46
loop 47
loop 48
loop 49
loop 50
loop 51
loop 52
loop 53
loop 54
loop 55
loop 56
loop 57
loop 58
loop 59
loop 60
loop 61
loop 62
loop 63
loop 64
loop 65
loop 66
loop 67
loop 68
loop 69
loop 70
loop 71
loop 72
loop 73
loop 74
loop 75
loop 76
loop 77
loop 78
loop 79
loop 80
loop 81
loop 82
loop 83
loop 84
loop 85
loop 86
loop 87
loop 88
loop 89
loop 90
loop 91
loop 92
loop 93
loop 94
loop 95
loop 96
loop 97
loop 98
loop 99
loop 100
loop 101
loop 102
loop 103
loop 104
loop 105
loop 106
loop 107
loop 108
loop 109
loop 110
loop 111
loop 112
loop 113
loop 114
loop 115
loop 116
loop 117
loop 118
loop 119
loop 120
loop 121
loop 122
loo

[{'properties.place': '353A ADMIRALTY DR',
  'geometry.coordinates': [103.8192487, 1.4505871]},
 {'properties.place': '353B ADMIRALTY DR',
  'geometry.coordinates': [103.8193846, 1.4502191]},
 {'properties.place': '353C ADMIRALTY DR',
  'geometry.coordinates': [103.8193166, 1.449724]},
 {'properties.place': '354A ADMIRALTY DR',
  'geometry.coordinates': [103.8188412, 1.4501124]},
 {'properties.place': '354B ADMIRALTY DR',
  'geometry.coordinates': [103.8186827, 1.4503887]},
 {'properties.place': '354C ADMIRALTY DR',
  'geometry.coordinates': [103.8182299, 1.4506118]},
 {'properties.place': '354D ADMIRALTY DR',
  'geometry.coordinates': [103.8188185, 1.4509385]},
 {'properties.place': '356A ADMIRALTY DR',
  'geometry.coordinates': [103.8169619, 1.449445]},
 {'properties.place': '356B ADMIRALTY DR',
  'geometry.coordinates': [103.8173555, 1.449223]},
 {'properties.place': '356C ADMIRALTY DR',
  'geometry.coordinates': [103.8163976, 1.4488366]},
 {'properties.place': '357 ADMIRALTY DR',
 

In [21]:
df2 = pd.DataFrame(coords)
df2

,properties.place,geometry.coordinates
0,353A ADMIRALTY DR,"[103.8192487, 1.4505871]"
1,353B ADMIRALTY DR,"[103.8193846, 1.4502191]"
2,353C ADMIRALTY DR,"[103.8193166, 1.449724]"
3,354A ADMIRALTY DR,"[103.8188412, 1.4501124]"
4,354B ADMIRALTY DR,"[103.8186827, 1.4503887]"
...,...,...
9507,181 YUNG SHENG RD,"[103.7219932, 1.3338632]"
9508,182 YUNG SHENG RD,"[103.7224675, 1.3338062]"
9509,183 YUNG SHENG RD,"[103.7227163, 1.3332724]"
9510,184 YUNG SHENG RD,"[103.7221742, 1.3334227]"


In [22]:
for coord, price in zip(coords, max_price_by_block):
    coord['properties.max_price'] = price
    

coords

[{'properties.place': '353A ADMIRALTY DR',
  'geometry.coordinates': [103.8192487, 1.4505871],
  'properties.max_price': 625000.0},
 {'properties.place': '353B ADMIRALTY DR',
  'geometry.coordinates': [103.8193846, 1.4502191],
  'properties.max_price': 588000.0},
 {'properties.place': '353C ADMIRALTY DR',
  'geometry.coordinates': [103.8193166, 1.449724],
  'properties.max_price': 620000.0},
 {'properties.place': '354A ADMIRALTY DR',
  'geometry.coordinates': [103.8188412, 1.4501124],
  'properties.max_price': 560000.0},
 {'properties.place': '354B ADMIRALTY DR',
  'geometry.coordinates': [103.8186827, 1.4503887],
  'properties.max_price': 590000.0},
 {'properties.place': '354C ADMIRALTY DR',
  'geometry.coordinates': [103.8182299, 1.4506118],
  'properties.max_price': 645000.0},
 {'properties.place': '354D ADMIRALTY DR',
  'geometry.coordinates': [103.8188185, 1.4509385],
  'properties.max_price': 664000.0},
 {'properties.place': '356A ADMIRALTY DR',
  'geometry.coordinates': [103.816

In [23]:
id = 1
for coord in coords:
    coord['geometry.type'] = 'Point'
    coord['id'] = id
    id += 1

coords

[{'properties.place': '353A ADMIRALTY DR',
  'geometry.coordinates': [103.8192487, 1.4505871],
  'properties.max_price': 625000.0,
  'geometry.type': 'Point',
  'id': 1},
 {'properties.place': '353B ADMIRALTY DR',
  'geometry.coordinates': [103.8193846, 1.4502191],
  'properties.max_price': 588000.0,
  'geometry.type': 'Point',
  'id': 2},
 {'properties.place': '353C ADMIRALTY DR',
  'geometry.coordinates': [103.8193166, 1.449724],
  'properties.max_price': 620000.0,
  'geometry.type': 'Point',
  'id': 3},
 {'properties.place': '354A ADMIRALTY DR',
  'geometry.coordinates': [103.8188412, 1.4501124],
  'properties.max_price': 560000.0,
  'geometry.type': 'Point',
  'id': 4},
 {'properties.place': '354B ADMIRALTY DR',
  'geometry.coordinates': [103.8186827, 1.4503887],
  'properties.max_price': 590000.0,
  'geometry.type': 'Point',
  'id': 5},
 {'properties.place': '354C ADMIRALTY DR',
  'geometry.coordinates': [103.8182299, 1.4506118],
  'properties.max_price': 645000.0,
  'geometry.typ

In [24]:
df_geo = pd.DataFrame(coords)
df_geo

,properties.place,geometry.coordinates,properties.max_price,geometry.type,id
0,353A ADMIRALTY DR,"[103.8192487, 1.4505871]",625000.0,Point,1
1,353B ADMIRALTY DR,"[103.8193846, 1.4502191]",588000.0,Point,2
2,353C ADMIRALTY DR,"[103.8193166, 1.449724]",620000.0,Point,3
3,354A ADMIRALTY DR,"[103.8188412, 1.4501124]",560000.0,Point,4
4,354B ADMIRALTY DR,"[103.8186827, 1.4503887]",590000.0,Point,5
...,...,...,...,...,...
9507,181 YUNG SHENG RD,"[103.7219932, 1.3338632]",590000.0,Point,9508
9508,182 YUNG SHENG RD,"[103.7224675, 1.3338062]",582000.0,Point,9509
9509,183 YUNG SHENG RD,"[103.7227163, 1.3332724]",570000.0,Point,9510
9510,184 YUNG SHENG RD,"[103.7221742, 1.3334227]",582000.0,Point,9511


In [25]:
import json
ok_json = json.loads(df_geo.to_json(orient='records'))
ok_json

[{'properties.place': '353A ADMIRALTY DR',
  'geometry.coordinates': [103.8192487, 1.4505871],
  'properties.max_price': 625000.0,
  'geometry.type': 'Point',
  'id': 1},
 {'properties.place': '353B ADMIRALTY DR',
  'geometry.coordinates': [103.8193846, 1.4502191],
  'properties.max_price': 588000.0,
  'geometry.type': 'Point',
  'id': 2},
 {'properties.place': '353C ADMIRALTY DR',
  'geometry.coordinates': [103.8193166, 1.449724],
  'properties.max_price': 620000.0,
  'geometry.type': 'Point',
  'id': 3},
 {'properties.place': '354A ADMIRALTY DR',
  'geometry.coordinates': [103.8188412, 1.4501124],
  'properties.max_price': 560000.0,
  'geometry.type': 'Point',
  'id': 4},
 {'properties.place': '354B ADMIRALTY DR',
  'geometry.coordinates': [103.8186827, 1.4503887],
  'properties.max_price': 590000.0,
  'geometry.type': 'Point',
  'id': 5},
 {'properties.place': '354C ADMIRALTY DR',
  'geometry.coordinates': [103.8182299, 1.4506118],
  'properties.max_price': 645000.0,
  'geometry.typ

In [29]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data


In [30]:
geojson = process_to_geojson(ok_json)
geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'place': '353A ADMIRALTY DR', 'max_price': 625000.0},
   'geometry': {'coordinates': [103.8192487, 1.4505871], 'type': 'Point'}},
  {'type': 'Feature',
   'properties': {'place': '353B ADMIRALTY DR', 'max_price': 588000.0},
   'geometry': {'coordinates': [103.8193846, 1.4502191], 'type': 'Point'}},
  {'type': 'Feature',
   'properties': {'place': '353C ADMIRALTY DR', 'max_price': 620000.0},
   'geometry': {'coordinates': [103.8193166, 1.449724], 'type': 'Point'}},
  {'type': 'Feature',
   'properties': {'place': '354A ADMIRALTY DR', 'max_price': 560000.0},
   'geometry': {'coordinates': [103.8188412, 1.4501124], 'type': 'Point'}},
  {'type': 'Feature',
   'properties': {'place': '354B ADMIRALTY DR', 'max_price': 590000.0},
   'geometry': {'coordinates': [103.8186827, 1.4503887], 'type': 'Point'}},
  {'type': 'Feature',
   'properties': {'place': '354C ADMIRALTY DR', 'max_price': 645000.0},
   'geometry': 

In [31]:
#Variable name
with open('map/geo-data.js', 'w') as outfile:
    outfile.write("var infoData = ")
#geojson output
with open('map/geo-data.js', 'a') as outfile:
    json.dump(geojson, outfile)
